In [ ]:
testdata = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3"""



In [ ]:
#use an enum for gridvalues
import re

class Sensor:
    def __init__(self,sensorPos:tuple[int,int],beaconPos:tuple[int,int]):
        self.sensorPos = sensorPos
        self.beaconPos = beaconPos
        self.manhattanDistance = abs(beaconPos[0] - sensorPos[0]) + abs(beaconPos[1] - sensorPos[1])

    def __str__(self):
        return str(self.sensorPos) +':' + str(self.manhattanDistance)

    def filledRangeInRow(self,y:int,maxSearchSpace:int)->tuple[int,int]:
        #return a range of x positions that represent where we know the cave isn't unknown
        dy = abs(y-self.sensorPos[1])
        if dy > self.manhattanDistance:
            return None
        else:
            xwidth = self.manhattanDistance - dy
            x = self.sensorPos[0]
            #part2 - limit to a range
            left = max(x - xwidth,0)
            right = min  (x + xwidth,maxSearchSpace)
            return (left,right)
    

        



class Cave:
    def __init__(self, input:str, maxSearchSpace):
        self.sensors = []
        self.maxSearchSpace = maxSearchSpace
        #parse the input
        pattern = '.*x=(-*[0-9]*).*y=(-*[0-9]*).*closest beacon is at x=(-*[0-9]*).*y=(-*[0-9]*)'
        for l in input.splitlines():
            match = re.search(pattern, l)
            sensorPos = (int(match.group(1)),int(match.group(2)))
            beaconPos = (int(match.group(3)),int(match.group(4)))
            self.sensors.append(Sensor(sensorPos,beaconPos))
    
    def findDistress(self)->int:
        #Need to loop through sensors
        #Will get a set of ranges back, that need to collapse into distinct overlapping ranges. Can then count each distinct range
        for y in range(self.maxSearchSpace):
            ranges:list[tuple[int,int]] = []
            distinctRanges:list[tuple[int,int]] = []

            for sensor in self.sensors:
                rr = sensor.filledRangeInRow(y,self.maxSearchSpace)
                #print('Sensor ' + str(sensor) + ' --Range: ' +str(rr))
                if rr != None:
                    ranges.append(rr)
            
            #we have risk (present in test data) that our distinct range are no longer distict - a new range might have bridged them.
            #We could solve this if we sort the ranges first, by the left hand value
            ranges.sort(key=lambda x: x[0])
            #print('--process ranges--')
            for left, right in ranges:
                #print('range: '+str((left,right)))
                overlapFound = False
                for i, (rl, rr) in enumerate(distinctRanges):
                    #rl, rr = distinctRanges[i]
                    if not overlapFound:
                        #is this overlapping?
                        if (left <= rl and right >= rl) or (right >= rr and left <= rr) or (left >= rl and right <= rr):
                            #extend this range
                            distinctRanges[i] = (min(left,rl),max(right,rr))
                            #print('Extending range: '+ str(rl)+':'+str(rr) + ' -> '+str(distinctRanges[i][0])+':'+str(distinctRanges[i][1]))
                            overlapFound = True
                if not overlapFound:
                    #print('New range: '+ str((left,right)))
                    distinctRanges.append((left,right))
        
    
            #now count the size of each range
            count = 0
            print('--')
            for rl, rr in distinctRanges:
                #print('Distinct Range: ' + str((rl,rr)))
                count += (rr - rl)
            possibleBeacons = self.maxSearchSpace - count - 1 #not sure when the one came from
            print('Row: ' +str(y).zfill(5) + '   possible beacons: ' + str(possibleBeacons))
            if possibleBeacons > 0:
                print('Possible beacon in row: ' + str(y))
                print(distinctRanges)
                #now need to find the x position
                x = 0
                for left, right in distinctRanges:
                    if x < left:
                        break
                    else:
                        x = right + 1
                print('Distress beacon at x:'+str(x)+' y:'+str(y))
                tuningFreq = x * 4000000 + y
                print('Tuning Freq = '+str(tuningFreq))

                break


    




#tests
testCave = Cave(testdata,20)
print(len(testCave.sensors))
print(testCave.sensors[0])
print('----')
testResult = testCave.findDistress()




In [ ]:
# using real puzzle input
input = open('day15input.txt').read()
cave = Cave(input,4000000)
result = cave.findDistress()